In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import collections
from nltk.stem import PorterStemmer 

In [2]:
! pip install transformers

     |████████████████████████████████| 501kB 2.9MB/s 
     |████████████████████████████████| 870kB 42.3MB/s 
     |████████████████████████████████| 3.7MB 35.2MB/s 
     |████████████████████████████████| 1.0MB 39.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=e95818fb24464b532122a0ddc41379ff1da9f2999ea2e0281bcb3aa8dcdb90cd
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [5]:
! nvidia-smi

Mon Mar  9 02:15:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from transformers import *

In [0]:
file_utterance = open("hw2_utterance_dev.txt", 'r')
utterances_dev = []
for line in file_utterance.readlines():
    utterances_dev.append(line[:-1])
file_tag = open("hw2_tags_dev.txt", "r")
tags_dev = []
for line in file_tag.readlines():
    tags_dev.append(line[:-1])

data = pd.read_csv('hw2_train.csv')
utterances_train = data['utterances']
tags_train = data['IOB Slot tags']
utterances_train = list(utterances_train) + utterances_dev
tags_train = list(tags_train) + tags_dev

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
def count_tag(data):
  tag_counter = collections.defaultdict(int)
  for tag in data:
      for t in tag.split():
          tag_counter[t] += 1
        
  tag_to_index = {'[PAD]' : 0}
  for k,v in sorted(tag_counter.items(), reverse=True, key=lambda kv:(kv[1], kv[0])):
      tag_to_index[k] = len(tag_to_index)

  index_to_tag = {}
  for k,v in tag_to_index.items():
      index_to_tag[v] = k
    
  return tag_to_index, index_to_tag
  
tag_to_index, index_to_tag = count_tag(tags_train)

In [8]:
def prepare(sentence, tag):
  MAX_LEN = 30
  sentence = sentence.split(" ")
  tag = tag.split(" ")
  while len(sentence) != len(tag):
    for i in range(len(sentence)):
      if sentence[i][0] == "'":
          temp = sentence[i-1] + sentence[i]
          sentence = sentence[:i-1] + [temp] + sentence[i+1:]
          break
  for i in range(len(sentence)):
    if sentence[i] == "'s":
      sentence[i] = "is"
#  while len(sentence) < 30:
#    sentence.append('[PAD]')
#    tag.append('[PAD]')
  sentence = ' '.join(sentence)
  sentence = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True, max_length=30, pad_to_max_length=True)).unsqueeze(0)
  while len(tag) < 30:
      tag.append('[PAD]')
  tag = torch.tensor([tag_to_index[i] for i in tag])
  return sentence.cuda(), tag.cuda()
prepare(utterances_train[3], tags_train[3])


(tensor([[ 101, 2424, 1996, 2931, 3883, 2013, 1996, 3185, 2016, 2003, 1996, 2158,
           102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0]], device='cuda:0'),
 tensor([1, 1, 1, 1, 1, 1, 1, 3, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0], device='cuda:0'))

In [0]:
class LSTMTagger(nn.Module):


    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)

    def forward(self, sentence):
        lstm_out, _ = self.lstm(sentence.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [16]:
bert = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=100 ).cuda()
loss_funtion = nn.CrossEntropyLoss()
model = LSTMTagger(100, 200, len(tag_to_index)).cuda()
optimizer = optim.Adam(model.parameters())
for epoch in range(30):
    total_loss = 0
    for i in range(len(utterances_train)):
        sentence, tag = prepare(utterances_train[i], tags_train[i])
        bert_outputs = bert(sentence)[0].view(30,-1)
        model.zero_grad()
        outputs = model(bert_outputs)
        loss = loss_funtion(outputs, tag)
        loss.backward()
        optimizer.step()
        total_loss += loss
    
    print("epoch=", epoch, "loss=", total_loss/len(data))
        

epoch= 0 loss= tensor(0.1884, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 1 loss= tensor(0.0898, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 2 loss= tensor(0.0630, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 3 loss= tensor(0.0446, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 4 loss= tensor(0.0289, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 5 loss= tensor(0.0195, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 6 loss= tensor(0.0138, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 7 loss= tensor(0.0116, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 8 loss= tensor(0.0079, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 9 loss= tensor(0.0066, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 10 loss= tensor(0.0059, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 11 loss= tensor(0.0043, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 12 loss= tensor(0.0042, device='cuda:0', grad_fn=<DivBackward0>)
epoch= 13 loss= tensor(0.0037, device='cuda:0', grad_fn=<DivBackward0>)
ep

In [17]:
def predict(sentence, tag):
    tag_length = len(tag.split(" "))
    sentence, tag = prepare(sentence, tag)
    bert_output = bert(sentence)[0].view(30,-1)
    output = model(bert_output)
    res = []
    for line in output:
        line = list(line)
        res.append(index_to_tag[line.index(max(line))])
    
    return res[:tag_length]
print(utterances_train[0])
print(predict(utterances_train[0], tags_train[0]))

who plays luke on star wars new hope
['O', 'O', 'B_char', 'O', 'B_movie', 'I_movie', 'I_movie', 'I_movie']


In [0]:
file_test = open('hw2_utterance_test.txt', 'r')
file_token = open('hw2_tokens_test.txt', 'r')
test_token = []
for line in file_token.readlines():
    test_token.append(line[:-1])

test = []
for line in file_test.readlines():
    test.append(line[:-1])
output = []
with torch.no_grad():
    for i in range(len(test)):
      try:
        res = predict(test[i], test_token[i])
        output.append(res)
      except:
        output.append(['O']*len(test_token[i].split(" ")))



In [0]:
with open('prediction.txt', 'w') as f:
    for line in output:
        f.write(' '.join(line))
        f.write('\n')

In [20]:
! python3 evaluation.py

F1 Score is 100.00%
